In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model, Sequential
import pandas as pd
import numpy as np
import model_training_utils as mtu

# svDB prep

In [3]:
svdb_data = pd.read_csv("data_sliced_tda/svDB.csv")
svdb_data = svdb_data[['betti_H1', 'ann']]

# Fix the lame type, string -> list
svdb_data['betti_H1'] = svdb_data['betti_H1'].apply(lambda x: mtu.fix_betti_string_svdb(x))
# Turn the list into a np.array of floats
svdb_data['betti_H1'] = svdb_data['betti_H1'].apply(lambda x: (np.array([float(num) for num in x])))

# mitDB prep

In [6]:
mitdb_data = pd.read_csv("data_sliced_tda/mitDB.csv")

mitdb_data['betti_H1'] = mitdb_data['betti'].apply(lambda x: np.array(mtu.fix_betti_string_mitdb(x)[1]))
mitdb_data = mitdb_data[['betti_H1', 'ann']]

# Random research

In [8]:
mitdb_data['betti_H1'][0].shape

(100,)

# Merging data and doing rest of the prep

In [9]:
concatenated_data = pd.concat([svdb_data, mitdb_data])
concatenated_data = concatenated_data.reset_index(drop=True)

In [14]:
X, y = concatenated_data['betti_H1'], concatenated_data['ann']

# Split to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=420)

# Encode the classes
y_train = to_categorical(np.array(y_train.to_list()))

# ????? Make a 3dim train data I guess
X_train = np.array(X_train.to_list())
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

# Model training

In [15]:
model = load_model('models_raw/4conv_1dens32.hdf5')
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2, verbose=1)
model.save("models_trained/both_svdb_mitdb_trained.hdf5")

2023-06-15 23:25:35.687494: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/10
1466/1467 [============================>.] - ETA: 0s - loss: 0.4408 - accuracy: 0.8279

2023-06-15 23:26:25.323988: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18769200 exceeds 10% of free system memory.


1467/1467 [==============================] - 51s 34ms/step - loss: 0.4408 - accuracy: 0.8279 - val_loss: 0.4020 - val_accuracy: 0.8356
Epoch 2/10
1467/1467 [==============================] - 50s 34ms/step - loss: 0.3996 - accuracy: 0.8399 - val_loss: 0.3807 - val_accuracy: 0.8476
Epoch 3/10
1467/1467 [==============================] - 49s 33ms/step - loss: 0.3844 - accuracy: 0.8452 - val_loss: 0.3679 - val_accuracy: 0.8521
Epoch 4/10
1467/1467 [==============================] - 52s 36ms/step - loss: 0.3746 - accuracy: 0.8494 - val_loss: 0.3605 - val_accuracy: 0.8537
Epoch 5/10
1467/1467 [==============================] - 52s 36ms/step - loss: 0.3691 - accuracy: 0.8506 - val_loss: 0.3582 - val_accuracy: 0.8539
Epoch 6/10
1467/1467 [==============================] - 57s 39ms/step - loss: 0.3637 - accuracy: 0.8525 - val_loss: 0.3546 - val_accuracy: 0.8559
Epoch 7/10
1467/1467 [==============================] - 56s 38ms/step - loss: 0.3601 - accuracy: 0.8538 - val_loss: 0.3502 - val_accura

AttributeError: 'History' object has no attribute 'save'

# Model testing and evaluating

In [17]:
# Encode the test classes
y_test = to_categorical(np.array(y_test.to_list()))

# ????? Make a 3dim train data I guess
X_test = np.array(X_test.to_list())
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

  49/1833 [..............................] - ETA: 5s - loss: 0.3392 - accuracy: 0.8552

2023-06-15 23:35:48.495063: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 23461600 exceeds 10% of free system memory.


1833/1833 [==============================] - 6s 3ms/step - loss: 0.3433 - accuracy: 0.8592
Test Loss: 0.34327471256256104
Test Accuracy: 0.8591911792755127
